In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-04-18 18:53:46--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  58.1MB/s    in 0.4s    

2024-04-18 18:53:47 (58.1 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [12]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! ")
  else:
    print("You haven't passed yet. Keep trying!")

books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

test_book_recommendation()

UnboundLocalError: local variable 'df_ratings' referenced before assignment

In [10]:
def get_recommends(book = ""):
    # import csv data into dataframes
    df_books = pd.read_csv(
        books_filename,
        encoding = "ISO-8859-1",
        sep=";",
        header=0,
        names=['isbn', 'title', 'author'],
        usecols=['isbn', 'title', 'author'],
        dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

    df_ratings = pd.read_csv(
        ratings_filename,
        encoding = "ISO-8859-1",
        sep=";",
        header=0,
        names=['user', 'isbn', 'rating'],
        usecols=['user', 'isbn', 'rating'],
        dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

    # remove users with less than 200 ratings and books with less than 100 ratings
    df_ratings = df_ratings.groupby('user').filter(lambda x: len(x) >= 200)
    df_ratings = df_ratings.groupby('isbn').filter(lambda x: len(x) >= 100)

    # merge ratings and books dataframes
    df = pd.merge(df_ratings, df_books, on='isbn')

    # create a pivot table
    df_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)

    # create a sparse matrix
    matrix = csr_matrix(df_pivot.values)

    # fit the nearest neighbors model
    model = NearestNeighbors(metric='cosine', algorithm='brute')
    model.fit(matrix)

    # get the index of the book
    idx = df_pivot.index.get_loc(book)

    # get the distances and indices of the nearest neighbors
    distances, indices = model.kneighbors(matrix[idx], n_neighbors=6)

    # get the recommended books
    recommended_books = [(df_pivot.index[i], dist) for i, dist in zip(indices[0][1:], distances[0][1:])]

    return [book, recommended_books]

In [11]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

UnboundLocalError: local variable 'df_ratings' referenced before assignment